In [418]:
import json
import pickle
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
from json_repair import repair_json

from itertools import chain
from collections import Counter
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer

### Read data

In [637]:
# Load the pickled list
with open("/Utilisateurs/umushtaq/emotion_analysis_comics/zeroshot/results/comics35_zs_pg_p2_mistral-7b-instruct-v0.3-bnb-4bit/results.pickle", 'rb') as f:
    
    results = pickle.load(f)

In [638]:
results

{'grounds': ["[['anger'], ['anger'], ['fear'], ['fear'], ['fear', 'sadness'], ['sadness'], ['anger'], ['surprise'], ['surprise'], ['fear', 'surprise']]",
  "[['fear'], ['anger'], ['surprise'], ['anger'], ['joy'], ['anger'], ['anger'], ['anger'], ['anger']]",
  "[['joy'], ['joy'], ['anger'], ['anger']]",
  "[['fear', 'surprise']]",
  "[['anger'], ['anger'], ['fear'], ['fear', 'surprise'], ['anger', 'joy'], ['anger']]",
  "[['sadness'], ['fear'], ['joy'], ['neutral'], ['surprise'], ['fear'], ['surprise'], ['joy'], ['surprise'], ['sadness', 'surprise']]",
  "[['surprise'], ['fear', 'sadness'], ['surprise'], ['surprise'], ['joy']]",
  "[['fear'], ['fear'], ['anger'], ['fear'], ['anger', 'surprise'], ['fear'], ['fear'], ['fear', 'sadness'], ['surprise'], ['surprise'], ['anger'], ['anger']]",
  "[['fear'], ['fear'], ['anger'], ['anger', 'fear', 'surprise'], ['fear', 'sadness'], ['fear', 'sadness'], ['anger'], ['anger'], ['surprise'], ['fear'], ['anger', 'surprise'], ['anger'], ['anger', 'sad

In [639]:
#all_labels = ["Anger", "Disgust", "Fear", "Sadness", "Surprise", "Joy"]

In [640]:
grounds = results['grounds']
predictions = results['predictions']

In [641]:
len(grounds), len(predictions)

(874, 874)

In [642]:
grounds

["[['anger'], ['anger'], ['fear'], ['fear'], ['fear', 'sadness'], ['sadness'], ['anger'], ['surprise'], ['surprise'], ['fear', 'surprise']]",
 "[['fear'], ['anger'], ['surprise'], ['anger'], ['joy'], ['anger'], ['anger'], ['anger'], ['anger']]",
 "[['joy'], ['joy'], ['anger'], ['anger']]",
 "[['fear', 'surprise']]",
 "[['anger'], ['anger'], ['fear'], ['fear', 'surprise'], ['anger', 'joy'], ['anger']]",
 "[['sadness'], ['fear'], ['joy'], ['neutral'], ['surprise'], ['fear'], ['surprise'], ['joy'], ['surprise'], ['sadness', 'surprise']]",
 "[['surprise'], ['fear', 'sadness'], ['surprise'], ['surprise'], ['joy']]",
 "[['fear'], ['fear'], ['anger'], ['fear'], ['anger', 'surprise'], ['fear'], ['fear'], ['fear', 'sadness'], ['surprise'], ['surprise'], ['anger'], ['anger']]",
 "[['fear'], ['fear'], ['anger'], ['anger', 'fear', 'surprise'], ['fear', 'sadness'], ['fear', 'sadness'], ['anger'], ['anger'], ['surprise'], ['fear'], ['anger', 'surprise'], ['anger'], ['anger', 'sadness'], ['anger']]",

In [643]:
predictions

['I apologize for the formatting issue in my previous response. Here\'s the corrected JSON output:\n\n[\n  ["anger", "sadness"],\n  ["anger", "sadness"],\n  ["fear"],\n  ["fear"],\n  ["anger"],\n  ["fear"],\n  ["anger"],\n  ["surprise"],\n  ["surprise", "fear"]\n]',
 'Identify applicable emotions from the following classes:\n"anger", "disgust", "fear", "sadness", "surprise", "joy", "neutral"\n\nAnd now analyze these utterances in a page:\n1: NO--  #GKKK…#\n2: #CHOMP!\n3: BY THE SKIN OF MATILDA!\n4: BLACKMANTASAURUS! WHAT KIND OF MONSTER EATS ITS OWN WARRIORS?\n5: #MUNCH! MUNCH!\n6: #PTOOO!\n7: VERY WELL.\n8: YOU WANT A FIGHT?\n9: YOU GOT ONE!\n\nOutput:\n{\n  "page_utterance_emotions": [\n    ["anger", "disgust"],\n    ["neutral"],\n    ["anger"],\n    ["fear", "disgust"],\n    ["neutral"],\n    ["neutral"],\n    ["neutral"],\n    ["anger"],\n    ["joy"]\n  ]\n}',
 '1: ["anger"]\n2: ["anger", "anticipation"]\n3: ["anger", "frustration"]\n4: ["excitement", "aggression"]',
 '1: "AARGH!"\

In [644]:
#grounds = [json.loads(x)["page_utterance_emotions"] for x in grounds]

In [645]:
#import re

In [646]:
# processed_preds = []
# bad_idx = []

# for i, input_string in enumerate(predictions):
    
#     json_content = re.search(r'\{.*\}', input_string, re.DOTALL)
    
#     if json_content:
#         json_string = json_content.group()
#         processed_preds.append(json_string)
#     else:    
#         bad_idx.append(i)
#         print(i)
        

In [647]:
predictions[66]

'"page_utterance_emotions": [["anger", "fear"], ["anger"], ["neutral"], ["doubt"], ["fear"], ["surprise", "pain"], ["sadness"]]'

In [648]:
bad_idx = []
processed_preds = []

for i,x in enumerate(predictions):
    try:
        y = repair_json(x)
        processed_preds.append(json.loads(y)["page_utterance_emotions"])
    except:
        print(i)
        bad_idx.append(i)

0
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
40
41
42
43
44
45
46
47
48
49
51
52
53
54
55
56
57
58
59
60
61
62
63
64
66
67
68
69
71
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
89
92
93
94
95
96
97
98
99
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
135
137
138
139
140
141
142
143
144
145
146
147
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
227
228
229
230
231
232
233
234
235
237
238
239
240
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
282
283
284
285
286
287
288
289
290
291
292

In [649]:
processed_preds

[[['anger', 'disgust'],
  ['neutral'],
  ['anger'],
  ['fear', 'disgust'],
  ['neutral'],
  ['neutral'],
  ['neutral'],
  ['anger'],
  ['joy']],
 [['anger', 'urgency'],
  ['concern', 'urgency'],
  ['annoyance'],
  ['frustration', 'sadness'],
  ['urgency'],
  ['fear', 'urgency']],
 [['fear'], ['neutral', 'surprise']],
 [['anger', 'fear'],
  ['curiosity', 'joy'],
  ['excitement', 'neutral'],
  ['neutral', 'anticipation'],
  ['anger', 'resignation']],
 [['anger', 'sadness'], ['neutral'], ['joy', 'pride'], ['sadness']],
 [['anger'], ['sadness'], ['sadness', 'fear']],
 [['neutral'],
  ['neutral', 'surprise'],
  ['neutral', 'anger'],
  ['neutral'],
  ['neutral', 'disgust'],
  ['neutral', 'surprise', 'disgust'],
  ['neutral', 'disgust'],
  ['neutral'],
  ['neutral'],
  ['neutral', 'surprise'],
  ['neutral', 'anger'],
  ['neutral', 'surprise'],
  ['neutral']],
 [['anger', 'surprise'],
  ['anger', 'pain'],
  ['fear'],
  ['joy'],
  ['fear'],
  ['anger', 'disgust'],
  ['hope'],
  ['fear'],
  ['ne

In [650]:
len(grounds), len(processed_preds)

(874, 43)

In [651]:
bad_idx.sort(reverse=True)

# Remove elements from 'grounds' at the specified indices
for idx in bad_idx:
    
    del grounds[idx]
    #del predictions[idx]

In [652]:
len(grounds), len(processed_preds)

(43, 43)

In [653]:
#predictions = [json.loads(x)["page_utterance_emotions"] for x in processed_preds]

In [654]:
#len(grounds), len(predictions)

In [655]:
grounds

["[['fear'], ['anger'], ['surprise'], ['anger'], ['joy'], ['anger'], ['anger'], ['anger'], ['anger']]",
 "[['fear'], ['surprise'], ['surprise'], ['fear', 'surprise'], ['fear'], ['anger']]",
 "[['sadness'], ['surprise']]",
 "[['joy'], ['surprise', 'joy'], ['surprise', 'joy'], ['fear', 'surprise']]",
 "[['joy'], ['joy'], ['fear', 'joy'], ['surprise']]",
 "[['anger'], ['anger', 'surprise'], ['fear']]",
 "[['surprise'], ['surprise'], ['surprise'], ['surprise', 'joy'], ['anger', 'disgust'], ['anger', 'surprise'], ['disgust', 'surprise'], ['disgust', 'joy'], ['fear', 'surprise'], ['joy'], ['joy'], ['joy'], ['surprise']]",
 "[['anger', 'fear'], ['anger', 'disgust', 'surprise'], ['fear', 'surprise'], ['surprise', 'joy'], ['disgust', 'fear', 'sadness'], ['fear', 'sadness'], ['neutral'], ['anger'], ['joy'], ['joy']]",
 "[['fear', 'surprise'], ['surprise'], ['surprise'], ['fear', 'surprise'], ['neutral'], ['joy'], ['joy'], ['neutral'], ['fear', 'surprise'], ['fear', 'surprise']]",
 "[['surprise']

In [656]:
import ast

In [657]:
grounds = [ast.literal_eval(x) for x in grounds]

In [658]:
grounds

[[['fear'],
  ['anger'],
  ['surprise'],
  ['anger'],
  ['joy'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger']],
 [['fear'],
  ['surprise'],
  ['surprise'],
  ['fear', 'surprise'],
  ['fear'],
  ['anger']],
 [['sadness'], ['surprise']],
 [['joy'], ['surprise', 'joy'], ['surprise', 'joy'], ['fear', 'surprise']],
 [['joy'], ['joy'], ['fear', 'joy'], ['surprise']],
 [['anger'], ['anger', 'surprise'], ['fear']],
 [['surprise'],
  ['surprise'],
  ['surprise'],
  ['surprise', 'joy'],
  ['anger', 'disgust'],
  ['anger', 'surprise'],
  ['disgust', 'surprise'],
  ['disgust', 'joy'],
  ['fear', 'surprise'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['surprise']],
 [['anger', 'fear'],
  ['anger', 'disgust', 'surprise'],
  ['fear', 'surprise'],
  ['surprise', 'joy'],
  ['disgust', 'fear', 'sadness'],
  ['fear', 'sadness'],
  ['neutral'],
  ['anger'],
  ['joy'],
  ['joy']],
 [['fear', 'surprise'],
  ['surprise'],
  ['surprise'],
  ['fear', 'surprise'],
  ['neutral'],
  ['joy'],
  ['joy'],
  ['neutral

In [659]:
processed_preds

[[['anger', 'disgust'],
  ['neutral'],
  ['anger'],
  ['fear', 'disgust'],
  ['neutral'],
  ['neutral'],
  ['neutral'],
  ['anger'],
  ['joy']],
 [['anger', 'urgency'],
  ['concern', 'urgency'],
  ['annoyance'],
  ['frustration', 'sadness'],
  ['urgency'],
  ['fear', 'urgency']],
 [['fear'], ['neutral', 'surprise']],
 [['anger', 'fear'],
  ['curiosity', 'joy'],
  ['excitement', 'neutral'],
  ['neutral', 'anticipation'],
  ['anger', 'resignation']],
 [['anger', 'sadness'], ['neutral'], ['joy', 'pride'], ['sadness']],
 [['anger'], ['sadness'], ['sadness', 'fear']],
 [['neutral'],
  ['neutral', 'surprise'],
  ['neutral', 'anger'],
  ['neutral'],
  ['neutral', 'disgust'],
  ['neutral', 'surprise', 'disgust'],
  ['neutral', 'disgust'],
  ['neutral'],
  ['neutral'],
  ['neutral', 'surprise'],
  ['neutral', 'anger'],
  ['neutral', 'surprise'],
  ['neutral']],
 [['anger', 'surprise'],
  ['anger', 'pain'],
  ['fear'],
  ['joy'],
  ['fear'],
  ['anger', 'disgust'],
  ['hope'],
  ['fear'],
  ['ne

In [660]:
bad_idx = []

for idx, (i,j) in enumerate(zip(grounds, processed_preds)):
    if len(i) != len(j):
        print(idx, len(i), len(j))
        bad_idx.append(idx)

3 4 5
9 1 5
12 5 6
13 12 11
20 1 4
21 15 14
22 8 7
24 10 11
25 11 12
30 12 8
34 2 5
35 4 3
36 1 2
38 1 5
39 1 5
41 5 1


In [661]:
len(bad_idx)

16

In [662]:
bad_idx.sort(reverse=True)

# Remove elements from 'grounds' at the specified indices
for idx in bad_idx:
    
    del grounds[idx]
    del processed_preds[idx]

In [663]:
len(grounds), len(processed_preds)

(27, 27)

In [664]:
grounds = [item for sublist in grounds for item in sublist]
predictions = [item for sublist in processed_preds for item in sublist]

In [665]:
#grounds = [[item for inner in outer for item in inner] for outer in grounds]
#predictions = [[item for inner in outer for item in inner] for outer in predictions]

In [666]:
len(grounds), len(predictions)

(173, 173)

In [667]:
grounds

[['fear'],
 ['anger'],
 ['surprise'],
 ['anger'],
 ['joy'],
 ['anger'],
 ['anger'],
 ['anger'],
 ['anger'],
 ['fear'],
 ['surprise'],
 ['surprise'],
 ['fear', 'surprise'],
 ['fear'],
 ['anger'],
 ['sadness'],
 ['surprise'],
 ['joy'],
 ['joy'],
 ['fear', 'joy'],
 ['surprise'],
 ['anger'],
 ['anger', 'surprise'],
 ['fear'],
 ['surprise'],
 ['surprise'],
 ['surprise'],
 ['surprise', 'joy'],
 ['anger', 'disgust'],
 ['anger', 'surprise'],
 ['disgust', 'surprise'],
 ['disgust', 'joy'],
 ['fear', 'surprise'],
 ['joy'],
 ['joy'],
 ['joy'],
 ['surprise'],
 ['anger', 'fear'],
 ['anger', 'disgust', 'surprise'],
 ['fear', 'surprise'],
 ['surprise', 'joy'],
 ['disgust', 'fear', 'sadness'],
 ['fear', 'sadness'],
 ['neutral'],
 ['anger'],
 ['joy'],
 ['joy'],
 ['fear', 'surprise'],
 ['surprise'],
 ['surprise'],
 ['fear', 'surprise'],
 ['neutral'],
 ['joy'],
 ['joy'],
 ['neutral'],
 ['fear', 'surprise'],
 ['fear', 'surprise'],
 ['neutral'],
 ['neutral'],
 ['anger', 'disgust'],
 ['anger', 'disgust'],
 [

In [668]:
predictions

[['anger', 'disgust'],
 ['neutral'],
 ['anger'],
 ['fear', 'disgust'],
 ['neutral'],
 ['neutral'],
 ['neutral'],
 ['anger'],
 ['joy'],
 ['anger', 'urgency'],
 ['concern', 'urgency'],
 ['annoyance'],
 ['frustration', 'sadness'],
 ['urgency'],
 ['fear', 'urgency'],
 ['fear'],
 ['neutral', 'surprise'],
 ['anger', 'sadness'],
 ['neutral'],
 ['joy', 'pride'],
 ['sadness'],
 ['anger'],
 ['sadness'],
 ['sadness', 'fear'],
 ['neutral'],
 ['neutral', 'surprise'],
 ['neutral', 'anger'],
 ['neutral'],
 ['neutral', 'disgust'],
 ['neutral', 'surprise', 'disgust'],
 ['neutral', 'disgust'],
 ['neutral'],
 ['neutral'],
 ['neutral', 'surprise'],
 ['neutral', 'anger'],
 ['neutral', 'surprise'],
 ['neutral'],
 ['anger', 'surprise'],
 ['anger', 'pain'],
 ['fear'],
 ['joy'],
 ['fear'],
 ['anger', 'disgust'],
 ['hope'],
 ['fear'],
 ['neutral'],
 ['neutral'],
 ['neutral'],
 ['joy', 'sadness'],
 ['neutral'],
 ['fear', 'curiosity'],
 ['surprise', 'joy', 'neutral'],
 ['neutral'],
 ['neutral'],
 ['neutral'],
 ['

In [669]:
# grounds = [item for sublist1 in grounds for sublist2 in sublist1 for item in sublist2]
# predictions = [item for sublist1 in predictions for sublist2 in sublist1 for item in sublist2]

In [670]:
len(grounds), len(predictions)

(173, 173)

In [671]:
mlb = MultiLabelBinarizer()

In [672]:
y_true_mhot = mlb.fit_transform(grounds)
y_pred_mhot = mlb.transform(predictions)

/Utilisateurs/umushtaq/.conda/envs/er_nb_env/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['amusement', 'anger: 0.1', 'annoyance', 'anticipation', 'apathy', 'caution', 'concern', 'confidence', 'contempt', 'curiosity', 'determination', 'disbelief', 'dominance', 'excitement: 0.8', 'frustration', 'gratitude', 'hope', 'hostility', 'joy: 0.9', 'pain', 'pride', 'recognition', 'sarcasm', 'surprise: 0.8', 'threat', 'urgency', 'worry'] will be ignored
  warnings.warn(


In [673]:
y_true_mhot.shape

(173, 7)

In [674]:
y_true_mhot.shape, y_pred_mhot.shape

((173, 7), (173, 7))

In [675]:
class_labels = mlb.classes_

In [676]:
class_labels

array(['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness',
       'surprise'], dtype=object)

In [677]:
print(classification_report(y_true_mhot, y_pred_mhot, target_names=class_labels, digits=3))

              precision    recall  f1-score   support

       anger      0.667     0.480     0.558        75
     disgust      0.188     0.273     0.222        11
        fear      0.536     0.312     0.395        48
         joy      0.143     0.095     0.114        21
     neutral      0.211     0.762     0.330        21
     sadness      0.091     0.105     0.098        19
    surprise      0.385     0.222     0.282        45

   micro avg      0.356     0.350     0.353       240
   macro avg      0.317     0.321     0.286       240
weighted avg      0.434     0.350     0.363       240
 samples avg      0.345     0.374     0.337       240



/Utilisateurs/umushtaq/.conda/envs/er_nb_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [313]:
with open("/Utilisateurs/umushtaq/emotion_analysis_comics/finetuning/saved_models/comics35_pg_old_Meta-Llama-3.1-70B-Instruct-bnb-4bit/classification_report_3.pickle", 'wb') as fh:
    
    pickle.dump(classification_report(y_true_mhot, y_pred_mhot, target_names=class_labels, digits=3, output_dict=True), fh)

In [ ]:
# grounds = results['grounds']
# predictions = results['predictions']

In [ ]:
# grounds = [json.loads(x)["page_utterance_emotions"] for x in grounds]
# predictions = [json.loads(x)["page_utterance_emotions"] for x in predictions]
#predictions = [x['page_utterance_emotions'] for x in predictions]

In [ ]:
#grounds

[[['surprise', 'joy'],
  ['joy'],
  ['surprise', 'joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['surprise'],
  ['joy'],
  ['joy'],
  ['neutral'],
  ['neutral']],
 [['neutral'],
  ['neutral'],
  ['anger', 'disgust'],
  ['anger', 'disgust'],
  ['neutral'],
  ['sadness'],
  ['sadness']],
 [['anger', 'sadness'],
  ['anger', 'sadness'],
  ['anger', 'sadness'],
  ['fear', 'surprise'],
  ['surprise'],
  ['joy'],
  ['anger', 'surprise'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['anger'],
  ['anger'],
  ['surprise', 'joy'],
  ['fear', 'sadness'],
  ['fear', 'sadness'],
  ['fear', 'surprise']],
 [['anger', 'disgust'],
  ['anger', 'disgust'],
  ['anger', 'disgust'],
  ['fear', 'sadness'],
  ['fear', 'sadness', 'surprise'],
  ['sadness'],
  ['sadness'],
  ['fear', 'sadness'],
  ['sadness', 'surprise'],
  ['sadness', 'surprise'],
  ['joy'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger', 'disgust'],
  ['joy'],
  ['joy'],
  ['surprise', 'joy'],
  ['surprise', 'joy'],
  ['anger', 'surprise'],
  ['anger', 

In [ ]:
#grounds_u = [[item for sublist in outer_list for item in sublist] for outer_list in grounds]

In [ ]:
#predictions_u = [[item for sublist in outer_list for item in sublist] for outer_list in predictions]

In [ ]:
#en(grounds_u), len(predictions_u)

(156, 156)

In [ ]:
#grounds_u

[['surprise',
  'joy',
  'joy',
  'surprise',
  'joy',
  'joy',
  'joy',
  'joy',
  'surprise',
  'joy',
  'joy',
  'neutral',
  'neutral'],
 ['neutral',
  'neutral',
  'anger',
  'disgust',
  'anger',
  'disgust',
  'neutral',
  'sadness',
  'sadness'],
 ['anger',
  'sadness',
  'anger',
  'sadness',
  'anger',
  'sadness',
  'fear',
  'surprise',
  'surprise',
  'joy',
  'anger',
  'surprise',
  'joy',
  'joy',
  'joy',
  'anger',
  'anger',
  'surprise',
  'joy',
  'fear',
  'sadness',
  'fear',
  'sadness',
  'fear',
  'surprise'],
 ['anger',
  'disgust',
  'anger',
  'disgust',
  'anger',
  'disgust',
  'fear',
  'sadness',
  'fear',
  'sadness',
  'surprise',
  'sadness',
  'sadness',
  'fear',
  'sadness',
  'sadness',
  'surprise',
  'sadness',
  'surprise',
  'joy',
  'anger',
  'anger',
  'anger',
  'anger',
  'disgust',
  'joy',
  'joy',
  'surprise',
  'joy',
  'surprise',
  'joy',
  'anger',
  'surprise',
  'anger',
  'surprise'],
 ['neutral', 'joy', 'joy'],
 ['neutral',
 

In [ ]:
#predictions_u

[['joy', 'joy', 'joy', 'joy', 'joy', 'joy', 'joy', 'joy', 'joy', 'joy', 'joy'],
 ['neutral',
  'neutral',
  'anger',
  'surprise',
  'anger',
  'surprise',
  'neutral',
  'anger',
  'disgust',
  'surprise',
  'anger',
  'disgust',
  'surprise'],
 ['anger',
  'anger',
  'sadness',
  'anger',
  'anger',
  'surprise',
  'anger',
  'surprise',
  'sadness',
  'anger',
  'surprise',
  'anger',
  'joy',
  'joy',
  'anger',
  'anger',
  'anger',
  'anger',
  'surprise',
  'joy',
  'joy'],
 ['anger',
  'anger',
  'anger',
  'sadness',
  'sadness',
  'sadness',
  'sadness',
  'anger',
  'surprise',
  'surprise',
  'joy',
  'anger',
  'anger',
  'anger',
  'anger',
  'anger',
  'anger',
  'anger',
  'joy',
  'anger',
  'anger'],
 ['joy', 'joy', 'joy'],
 ['sadness',
  'fear',
  'surprise',
  'sadness',
  'fear',
  'sadness',
  'joy',
  'sadness',
  'fear',
  'sadness',
  'sadness',
  'joy',
  'sadness',
  'fear',
  'sadness',
  'joy',
  'sadness',
  'joy',
  'joy',
  'sadness',
  'joy',
  'anger',

In [ ]:
#len(grounds_u), len(predictions_u)

(156, 156)

In [ ]:
# from sklearn.preprocessing import MultiLabelBinarizer
# mlb = MultiLabelBinarizer()

In [ ]:
#y_true_mhot = mlb.fit_transform(grounds_u)

In [ ]:
#y_true_mhot.shape

(156, 7)

In [ ]:
#y_true_mhot

array([[0, 0, 0, ..., 1, 0, 1],
       [1, 1, 0, ..., 1, 1, 0],
       [1, 0, 1, ..., 0, 1, 1],
       ...,
       [1, 0, 0, ..., 0, 1, 0],
       [1, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0]])

In [ ]:
#y_pred_mhot = mlb.transform(predictions_u)

In [ ]:
#y_pred_mhot.shape

(156, 7)

In [ ]:
#y_pred_mhot

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 1, 0, 1],
       [1, 0, 0, ..., 0, 1, 1],
       ...,
       [1, 0, 1, ..., 0, 1, 1],
       [0, 0, 0, ..., 0, 1, 1],
       [1, 0, 0, ..., 0, 0, 0]])

In [ ]:
#class_labels = mlb.classes_

In [ ]:
#class_labels

array(['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness',
       'surprise'], dtype=object)

In [ ]:
#print(classification_report(y_true_mhot, y_pred_mhot, target_names=class_labels, digits=3))

              precision    recall  f1-score   support

       anger      0.929     0.778     0.847       117
     disgust      0.318     0.250     0.280        28
        fear      0.756     0.626     0.685        99
         joy      0.855     0.678     0.756        87
     neutral      0.778     0.311     0.444        45
     sadness      0.750     0.734     0.742        94
    surprise      0.948     0.879     0.912       124

   micro avg      0.829     0.692     0.754       594
   macro avg      0.762     0.608     0.667       594
weighted avg      0.825     0.692     0.746       594
 samples avg      0.835     0.692     0.727       594

